### Load the API Keys

In [ ]:
from dotenv import load_dotenv

# from langchain_anthropic import ChatAnthropic
# from langchain_aws import ChatBedrock
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

True

### Load Model

In [ ]:
### for this notebook, we use the Google Generative AI model
# Uncomment the following line to use Anthropic's Claude model instead or another model

# llm = ChatAnthropic(model="claude-3.5-sonnet-20240620", anthropic_api_key="...")
# llm = ChatBedrock( model_id=anthropic.claude-3-5-sonnet-20241022-v2:0, region_name="us-west-2")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

### Simple Sequential Chain:

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("Write a 3 line poem on {topic}")
chain = prompt | llm | StrOutputParser()
result = chain.invoke({"topic": "autumn"})
print(result)

Golden leaves are falling down,
Crisp air whispers through the town,
Nature's beauty wears a crown.


### Sequential Chain

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Generate a 2-point summary from the following text \n {text}',
    input_variables=['text']
)
parser = StrOutputParser()
chain = prompt1 | llm | parser | prompt2 | llm | parser
result = chain.invoke({'topic': 'Global Recession'})
print(result)

Here's a 2-point summary of the provided text:

*   **Global recessions are significant declines in worldwide economic activity, characterized by widespread contractions in GDP, trade, investment, and employment, often triggered by financial crises, demand/supply shocks, policy mistakes, or structural imbalances.**
*   **Effective policy responses, including coordinated monetary and fiscal measures, financial sector reforms, and international cooperation, are crucial to mitigating the impacts of global recessions and promoting a sustainable recovery, especially given the current economic climate of high inflation, geopolitical tensions, and supply chain disruptions.**


In [5]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

### Parallel Chain

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

# Initialize model and parser
parser = StrOutputParser()

# Step 1a: Summary prompt
summary_prompt = PromptTemplate.from_template("Summarize this topic:\n{text}")
summary_chain = summary_prompt | llm | parser

# Step 1b: Fun fact prompt
fact_prompt = PromptTemplate.from_template("Give a fun fact about:\n{text}")
fact_chain = fact_prompt | llm | parser

# Run both in parallel
parallel = RunnableParallel({
    "summary": summary_chain,
    "fact": fact_chain
})

# Step 2: Merge both outputs
merge_prompt = PromptTemplate.from_template(
    "Here is a short summary and a fun fact combined:\nSummary: {summary}\nFun Fact: {fact}"
)
merge_chain = merge_prompt | llm | parser

# Final full chain
chain = parallel | merge_chain

# Sample input
text = "Photosynthesis is the process by which green plants use sunlight to make food from carbon dioxide and water."

# Run
result = chain.invoke({"text": text})
print(result)

This is a great, memorable way to present the information! The summary is concise and accurate, and the fun fact is engaging and humorous. The "plant farts" analogy is certainly attention-grabbing and makes the concept of oxygen being a byproduct of photosynthesis much more memorable. Well done!


In [8]:
# Visualize the chain
chain.get_graph().print_ascii()

                    +-----------------------------+                    
                    | Parallel<summary,fact>Input |                    
                    +-----------------------------+                    
                       ***                   ***                       
                   ****                         ****                   
                 **                                 **                 
    +----------------+                          +----------------+     
    | PromptTemplate |                          | PromptTemplate |     
    +----------------+                          +----------------+     
             *                                           *             
             *                                           *             
             *                                           *             
+------------------------+                  +------------------------+ 
| ChatGoogleGenerativeAI |                  | ChatGoogleGenerati

### Conditional Chain:

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from pydantic import BaseModel, Field
from typing import Literal

# llm = ChatOpenAI()
text_parser = StrOutputParser()
# 1. Define structured output schema
class Feedback(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(
        description="Classify the feedback sentiment"
    )
structured_parser = PydanticOutputParser(pydantic_object=Feedback)
# 2. Classification prompt (with parser instruction)
classification_prompt = PromptTemplate(
    template=(
        "Analyze the feedback and classify it as 'positive' or 'negative'.\n"
        "Feedback: {feedback}\n\n"
        "{format_instruction}"
    ),
    input_variables=["feedback"],
    partial_variables={"format_instruction": structured_parser.get_format_instructions()}
)
# Chain: classify → structured output
classifier_chain = classification_prompt | llm | structured_parser
# 3. Conditional branches for response generation
prompt1 = PromptTemplate.from_template(
          "Write a warm thank-you message for this feedback:\n{feedback}"
           )
positive_response = prompt1 | llm | text_parser
prompt2 = PromptTemplate.from_template(
          "Write a polite and helpful apology response for this feedback:\n{feedback}"
          )
negative_response = prompt2 | llm | text_parser
# 4. Conditional logic using RunnableBranch
response_branch = RunnableBranch(
    (lambda x: x.sentiment == "positive", positive_response),
    (lambda x: x.sentiment == "negative", negative_response),
    RunnableLambda(lambda _: "Sentiment could not be classified.")
)
# 5. Final chain: classify → choose response
chain = classifier_chain | response_branch
# Example usage
feedback_text = "The product arrived late and the box was damaged."
print(chain.invoke({"feedback": feedback_text}))

Okay, I understand. I'm very sorry to hear that you had a negative experience. I truly value your feedback, as it helps me learn and improve. Could you please provide me with more details about what went wrong? Knowing the specifics will allow me to address the issue and prevent it from happening again in the future.

In the meantime, please accept my sincere apologies. I am committed to doing better.


### Router Chains

In [13]:
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# Base model: Gemini Flash
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Output parser
parser = StrOutputParser()

# Prompt for Python questions
python_prompt = PromptTemplate.from_template(
    "You are a Python expert. Answer the following question in detail:\n{question}"
)
python_chain = python_prompt | llm | parser

# Prompt for SQL questions
sql_prompt = PromptTemplate.from_template(
    "You are a SQL expert. Explain this SQL-related query in detail:\n{question}"
)
sql_chain = sql_prompt | llm | parser

# Router logic based on keywords
def route_question(input):
    q = input["question"].lower()
    if "python" in q:
        return 0  # Route to Python chain
    elif "sql" in q:
        return 1  # Route to SQL chain
    else:
        return 2  # Fallback response

# Fallback chain
fallback_chain = RunnableLambda(
    lambda x: "Sorry, I can only answer questions about Python or SQL at the moment."
)

# Router chain using RunnableBranch
router_chain = RunnableBranch(
    (lambda x: route_question(x) == 0, python_chain),
    (lambda x: route_question(x) == 1, sql_chain),
    fallback_chain
)

# Run it
print(router_chain.invoke({"question": "How does a Python generator work?"}))


Okay, let's dive deep into how Python generators work. They are a powerful and memory-efficient way to create iterators.

**What is a Generator?**

At its core, a generator is a special type of function that produces a sequence of values using the `yield` keyword.  Unlike a regular function that executes to completion and returns a single value, a generator *pauses* its execution each time it encounters a `yield` and returns a value to the caller. It remembers its state so that it can resume execution from where it left off the next time a value is requested.

**Key Characteristics and Differences from Regular Functions:**

1. **`yield` Keyword:** The presence of the `yield` keyword is the defining characteristic of a generator function.  Instead of `return`, generators use `yield` to produce values.

2. **Iterator Protocol:** Generators automatically implement the iterator protocol.  This means they have `__iter__()` and `__next__()` methods (either implicitly or explicitly). This all

In [14]:
print(router_chain.invoke({"question": "What is a SQL JOIN?"}))


Okay, let's dive into the world of SQL JOINs. I'll provide a comprehensive explanation, covering the concept, the different types, and practical considerations.

**What is a SQL JOIN?**

At its core, a SQL JOIN clause is used to **combine rows from two or more tables** based on a related column between them. Imagine you have two separate lists of information, and you want to put them together based on something they have in common (like a customer ID). That's what a JOIN does.  It's a fundamental operation for relational databases because data is often spread across multiple tables to maintain data integrity and avoid redundancy.

**Why Use JOINs?**

*   **Data Normalization:**  JOINs are essential when your database adheres to normalization principles. Normalization reduces data duplication by storing related information in separate tables.  JOINs allow you to bring this related data back together for analysis, reporting, or application use.
*   **Combining Information:** You might ha

In [15]:
print(router_chain.invoke({"question": "Tell me about cloud computing."}))

Sorry, I can only answer questions about Python or SQL at the moment.
